**Neural networks**는 **torch.nn** 패키지를 이용해서 만들 수 있다. 

이미 **autograd**에 대해서 살펴봤다. **nn**은 모델을 정의하고 차별화하기 위해서 **autograd**에 의존한다. **nn.Module**은 layer들과 출력을 반환하는 **forward(input)**을 포함하고 있다. 

일반적인 neural network의 학습 절차는 다음과 같다. 

1. 학습 가능한 매개변수(또는 가중치)가 있는 신경망을 정의
2. 입력 데이터셋에 따라서 반복
3. network를 통해서 input을 처리하고
4. 손실을 계산
5. gradient를 네트워크의 변수들로 다시 전파
6. 네트워크의 가중치를 업데이트:  **weight = weight - learning_rate*gradient**

# Define the network

In [0]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Net(nn.Module):
  
  def __init__(self):
    super(Net, self).__init__()
    
    self.conv1 = nn.Conv2d(1, 6, 3) #(input img channel, output channel, nxn kernel)
    self.conv2 = nn.Conv2d(6, 16, 3)
    
    self.fc1 = nn.Linear(16*6*6, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    # Max pooling over a (2x2) window, squre라면 single number로도 가능
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
  def num_flat_features(self, x):
    size = x.size()[1:] # all dimensions except the batch dimension
    num_features = 1 
    for s in size:
      num_features *= s 
    return num_features
  
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [0]:
# learnable parameters 

params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [0]:
input = torch.rand(1, 1, 32, 32) #(nSample, nChannels, Height, Width)

out = net(input)

print(out)

tensor([[ 0.1080, -0.0545,  0.0185,  0.0997, -0.0699, -0.0014,  0.0305,  0.1419,
         -0.0366, -0.0113]], grad_fn=<AddmmBackward>)


무작위 gradient가 있는 backprop과 모든 매개 변수의 gradient buffer를 제로화

In [0]:
net.zero_grad()
out.backward(torch.rand(1,10))

**torch.nn** 패키지는 미니 배치만을 지원한다. 전체 **torch.nn**가 단일 샘플이 아닌 미니 배치 샘플 임력만을 지원한다.

예를 들면, **torch.nn.Conv2d**는 (nSample x nChannels x Height x Width)의 4D tensor를 이용한다. 

단일 샘플을 사용하는 경우, 가짜 batch dimension을 추가하기 위한 **input.unsqueeze(0)**을 사용한다. 

## Loss Function

loss function은 (output, target) 형태의 입력값을 가진다. 
output과 target 사이의 거리가 얼마나 먼가를 계산한다. 

nn package에는 많은 종류의 loss function이 있다. 가장 간단한 loss function 중 하나는 **nn.MSELoss**이며 mean squred error를 loss 값으로 사용한다. 

In [0]:
output = net(input)
target = torch.rand(10) # a dummy target, for example.
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)

print(loss)

tensor(0.3760, grad_fn=<MseLossBackward>)


In [0]:
print(loss.grad_fn) #MSELOSS
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

##Backprop

에러를 역전파 하기 위해서 우리가 할 수 있는 것은 **loss.backward()**하는 것이다. 

존재하는 gradients들을 먼저 초기화 한다. 그렇지 않으면 gradients들이 이미 있던 gradients들과 섞이면서 쌓이게 된다. 

conv1의 bias gradients 들을 backward 전, 후 어떻게 바뀌는지 살펴보자. 

In [0]:
net.zero_grad()

In [0]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0007,  0.0057,  0.0017, -0.0069, -0.0030,  0.0052])


이것은 loss 가 neural network에서 어떻게 사용되는지를 보여준다.

## Update the weights

**weight = weight - learning_rate x gradient**

이 업데이트 식은 python에서 다음과 같이 계산할 수 있다.

In [0]:
learning_rate = 0.01

for f in net.parameters():
  f.data.sub_(f.grad.data*learning_rate)

하지만 신경망을 사용하면 SGD, Nesterov-SGD, Adam, RMSProp 등과 같은 다양한 업데이트 규칙을 사용할 수 있다. **torch.optim** 패키지에서 위 업데이트 방법들을 간단하게 사용할 수 있다. 

In [0]:
import torch.optim as optim

# optimizer 만들기
optimizer = optim.SGD(net.parameters(), lr = 0.01)

optimizer.zero_grad() # initialize the gradient buffers

output = net(input) # get output from the net
loss = criterion(output, target) # calculate the distance between output and target
loss.backward() # backpropagation
optimizer.step() # update the weights